In [11]:
#dict
data = open('words.txt', 'r')
words = []
for line in data:
    line = line.strip()
    if len(line) == 1:
        continue
    
    words.append(line)

print(len(words))

words = words[:90]

99119


In [ ]:
words

In [12]:
from FST import FST as FST

palavras_de_teste = ['mon', 'thurs', 'tues','tye', 'zon']

palavras_de_teste1 = ["A's", "AA's", "AB's", "ABM's", "AC's", "ACTH's"]

palavras_de_teste2 = ['apr', 'aug', 'mar', 'may']

palavras_de_teste3 = ['aab111','aab222','aab333','ab111', 'ab222', 'ab333']

palavras_de_teste4 = ['a', 'aa', 'aaa', 'aaaa', 'aaaaa', 'aaaaaa', 'aaaaaaa', 'aaaaaaaa', 'aaaaaaaaa', 'aaaaaaaaaa', 'b', 'ba']

palavras_de_teste5 = ['Actaeon', 'Acton', 'Acts']

fst = FST(words=words)

In [6]:
fst.imprimir()
fst.imprimir_inverso()

from graphviz import render

arquivo = 'fst.dot'
arquivo2 = 'fst_inverso.dot'

render('dot', 'png', arquivo)
render('dot', 'png', arquivo2)

Número de nós: 163


'fst_inverso.dot.png'

In [13]:
def get_matches(word):
    resp = fst.checar_combinacoes(word)
    #print("Total de palavras encontradas: " + str(len(resp)) + "\n")
    return resp

In [14]:
def get_matches_for(word):
    matches = []
    cont = 0
    for w in words:
        if w.startswith(word):
            matches.append(w)
            cont += 1
    #print("Total de palavras encontradas: " + str(len(matches)) + "\n")
    return matches

In [15]:
def get_matches_binary_search(word):
    num_words = len(words)
    matches = []
    cont = 0
    ini = 0
    fim = num_words - 1
    atual = 0
    
    while ini <= fim:
        atual = (ini + fim) // 2
        if (words[atual].startswith(word)):
            fim = -1
        elif (words[atual] > word):
            fim = atual - 1
        else:
            ini = atual + 1
    
    while words[atual-1].startswith(word):
        atual -= 1

    while words[atual].startswith(word) and atual < len(words)-1:
        matches.append(words[atual])
        cont += 1
        atual += 1

    if atual == len(words)-1:
        if words[atual].startswith(word):
            matches.append(words[atual])
            cont += 1

    #print("Total de palavras encontradas: " + str(len(matches)) + "\n")
    return matches

In [ ]:
import sys
print(sys.getsizeof(words))
print(sys.getsizeof(get_matches))
print(sys.getsizeof(get_matches_for))
print(sys.getsizeof(get_matches_binary_search))

In [16]:
teste = ""
match1 = get_matches(teste)
match2 = get_matches_for(teste)

print("Palavras encontradas pelo método checar_combinacoes: ", len(match1))
print("Palavras encontradas pelo método get_matches_for: ", len(match2))

for m in match1:
    if m not in match2:
        print("Erro: " + m + " não está na lista de palavras encontradas pelo método get_matches_for")

for m in match2:
    if m not in match1:
        print("Erro: " + m + " não está na lista de palavras encontradas pelo método checar_combinacoes")

Palavras encontradas pelo método checar_combinacoes:  90794
Palavras encontradas pelo método get_matches_for:  99119
Erro: Actaeon não está na lista de palavras encontradas pelo método checar_combinacoes
Erro: Acton não está na lista de palavras encontradas pelo método checar_combinacoes
Erro: Amoco não está na lista de palavras encontradas pelo método checar_combinacoes
Erro: Amoco's não está na lista de palavras encontradas pelo método checar_combinacoes
Erro: Andersen não está na lista de palavras encontradas pelo método checar_combinacoes
Erro: Andersen's não está na lista de palavras encontradas pelo método checar_combinacoes
Erro: Angelica não está na lista de palavras encontradas pelo método checar_combinacoes
Erro: Antonia não está na lista de palavras encontradas pelo método checar_combinacoes
Erro: Antonia's não está na lista de palavras encontradas pelo método checar_combinacoes
Erro: Antoninus não está na lista de palavras encontradas pelo método checar_combinacoes
Erro: An

In [ ]:
print(len(get_matches("a")))

In [ ]:
print(len(get_matches_for("a")))

In [ ]:
print(get_matches_binary_search("A"))

In [ ]:
prefixos = set()

for word in words:
    for i in range(1, len(word)):
        prefixo = word[:i]

        if prefixo not in prefixos:
            len1 = len(get_matches(prefixo))
            try:
                len2 = len(get_matches_binary_search(prefixo))
            except:
                print(prefixo, "erro")
            

            if len2 != len1:
                print(prefixo, len1, len2)

        prefixos.add(prefixo)



In [ ]:
import tkinter as tk
import timeit

class AutocompleteText(tk.Text):
    def __init__(self, *args, **kwargs):
        self.callback = kwargs.pop("autocomplete", None)
        self.comparativo1 = kwargs.pop("comparativo1", None)
        super().__init__(*args, **kwargs)
        self.bind("<Any-KeyRelease>", self._autocomplete)
        self.bind("<Tab>", self._handle_tab)

    def _handle_tab(self, event):
        tag_ranges= self.tag_ranges("autocomplete")
        if tag_ranges:
            self.mark_set("insert", tag_ranges[1])
            self.tag_remove("sel", "1.0", "end")
            self.tag_remove("autocomplete", "1.0", "end")

            return "break"

    def _autocomplete(self, event):
        if event.char and self.callback:
            word = self.get("insert-1c wordstart", "insert-1c wordend")
            inicio = timeit.default_timer()
            matches = self.callback(word)
            fim = timeit.default_timer()

            inicio2 = timeit.default_timer()
            matches2 = self.comparativo1(word)
            fim2 = timeit.default_timer()

            text_tempo1["text"] = "Tempo de atualização I: " + str(round((fim - inicio)*1000,3)) + " ms"

            text_tempo2["text"] = "Tempo de atualização II: " + str(round((fim2 - inicio2)*1000,3)) + " ms"

            if matches:
                remainder = matches[0][len(word):]
                insert = self.index("insert")
                self.insert(insert, remainder, ("sel", "autocomplete"))
                self.mark_set("insert", insert)
            
            texto = "Outras Sugestões: \n"
            for match in matches[1:]:
                texto += match + "\n"
            texto_outras_opcoes["text"] = texto
            

root = tk.Tk()

# titulo
root.title("Autocomplete")

# tamanho da janela
root.geometry("300x650")

# Label
label = tk.Label(root, text="Digite uma palavra: ")
label.pack(padx=10, pady=10)

# texto
text = AutocompleteText(root, autocomplete=get_matches, comparativo1 = get_matches_binary_search)
text.pack(padx=10, pady=10)

# Outras opções
texto_outras_opcoes = tk.Label(root, text="Outras opções: ")
texto_outras_opcoes.pack(padx=10, pady=10)

# tempo de atualização
text_tempo1 = tk.Label(root, text="Tempo de atualização I: ")
text_tempo1.pack(padx=10, pady=5)

# tempo de atualização
text_tempo2 = tk.Label(root, text="Tempo de atualização II: ")
text_tempo2.pack(padx=10, pady=5)

root.mainloop()

In [ ]:
class Teste:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __eq__(self, other):
        return self.x == other.x

    def __hash__(self):
        return hash((self.x, self.y))

# Exemplo de uso
objeto1 = Teste(1, 2)
objeto2 = Teste(1, 2)
objeto3 = Teste(1, 4)

conjunto = set([objeto1, objeto2, objeto3])

print(len(conjunto))

print(objeto1 == objeto3)